### Preparation

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
"""
!unzip '/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data.zip' -d '/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data'
"""

Archive:  /content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data.zip
  inflating: /content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_label.csv  
  inflating: /content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/__MACOSX/Master project data/._data_label.csv  
replace /content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_org_image/GfIraxmb.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_org_image/GfIraxmb.png  
  inflating: /content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/__MACOSX/Master project data/data_org_image/._GfIraxmb.png  
replace /content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_org_image/P9giY4n0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: a

In [2]:
import numpy as np
from collections import Counter

In [3]:
!pip install torchtoolbox

     |████████████████████████████████| 84 kB 2.4 MB/s 
     |████████████████████████████████| 3.1 MB 56.9 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 60.6 MB/s 
     |████████████████████████████████| 3.3 MB 42.3 MB/s 
     |████████████████████████████████| 596 kB 70.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
! pip install -U albumentations

     |████████████████████████████████| 102 kB 6.6 MB/s 
     |████████████████████████████████| 47.6 MB 32 kB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
def partition_data(file='/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_label.csv', ratios=[0.70, 0.20, 0.10], out_path='/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/', uniform=False):
    """
    Partition data into training, validation and test data. Saves partitioned data
    to three .csv files.
    """

    # Define ratios for traning, validation and test
    ratio_train = ratios[0]
    ratio_validation = ratios[1]
    ratio_test = ratios[2]

    # Import data
    data = np.genfromtxt(file, skip_header=1, dtype='str', delimiter=',')

    # Divide by image and label
    imgs = data[:,0]
    lbls = data[:,1]

    idxs_shuffle = np.random.permutation(len(lbls))
    imgs = imgs[idxs_shuffle]
    lbls = lbls[idxs_shuffle]

    # Print info
    print('NUMBER OF SAMPLES')
    samples_per_class(lbls)
    print('')

    # If uniformly many classes (do not use all data however)
    if uniform:
        # Find indices of each class
        idxs0 = np.where(lbls=='0')
        idxs1 = np.where(lbls=='1')
        idxs2 = np.where(lbls=='2')

        # Find number of samples for each class
        N0 = len(idxs0[0])
        N1 = len(idxs1[0])
        N2 = len(idxs2[0])
        N = min(N0, N1, N2)

        # Divide data by classes
        imgs0 = imgs[idxs0]
        lbls0 = lbls[idxs0]
        imgs1 = imgs[idxs1]
        lbls1 = lbls[idxs1]
        imgs2 = imgs[idxs2]
        lbls2 = lbls[idxs2]

        # Shuffle order
        idxs_shuffle0 = np.random.permutation(N0)
        idxs_shuffle1 = np.random.permutation(N1)
        idxs_shuffle2 = np.random.permutation(N2)
        imgs0 = imgs0[idxs_shuffle0]
        lbls0 = lbls0[idxs_shuffle0]
        imgs1 = imgs1[idxs_shuffle1]
        lbls1 = lbls1[idxs_shuffle1]
        imgs2 = imgs2[idxs_shuffle2]
        lbls2 = lbls2[idxs_shuffle2]

        # Take samples from each class
        imgs = []
        lbls = []
        for n in range(N):
            imgs.append(imgs0[n])
            lbls.append(lbls0[n])
            imgs.append(imgs1[n])
            lbls.append(lbls1[n])
            imgs.append(imgs2[n])
            lbls.append(lbls2[n])

    # Divide into training, validation and test data sets
    idxs_train = int(ratio_train*len(imgs))
    idxs_validation = int((ratio_train+ratio_validation)*len(imgs))
    imgs_train = imgs[0:idxs_train]
    imgs_validation = imgs[idxs_train:idxs_validation]
    imgs_test = imgs[idxs_validation:]
    lbls_train = lbls[0:idxs_train]
    lbls_validation = lbls[idxs_train:idxs_validation]
    lbls_test = lbls[idxs_validation:]

    # Save to files
    np.savetxt(out_path+'data_labels_train.csv', np.vstack((imgs_train, lbls_train)).T, delimiter=',', fmt='%s')
    np.savetxt(out_path+'data_labels_validation.csv', np.vstack((imgs_validation, lbls_validation)).T, delimiter=',', fmt='%s')
    np.savetxt(out_path+'data_labels_test.csv', np.vstack((imgs_test, lbls_test)).T, delimiter=',', fmt='%s')

    # Print information
    print('PARTITIONED DATA')
    print('  Training:    ' + str(int(100*ratios[0]))+'%, ' + str(len(lbls_train)) + ' samples')
    samples_per_class(lbls_train)
    print('')
    print('  Validation:  ' + str(int(100*ratios[1]))+'%, ' + str(len(lbls_validation)) + ' samples')
    samples_per_class(lbls_validation)
    print('')
    print('  Testing:     ' + str(int(100*ratios[2]))+'%, ' + str(len(lbls_test)) + ' samples')
    samples_per_class(lbls_test)
    print('')
    print('  Total:       ' + str(len(lbls_train)+len(lbls_validation)+len(lbls_test)))


def samples_per_class(lbls):
    counter = Counter(lbls)
    keys = counter.keys()
    values = counter.values()
    pairs = sorted(zip(keys, values))
    sum = 0
    output_format = '    Class {:<1}: {:>5}   {:>3}%'
    for pair in pairs:
        sum += pair[1]
    for pair in pairs:
        print(output_format.format(pair[0], pair[1], int(100*pair[1]/sum)))
    print('    Total:   {:>5}'.format(sum))

In [ ]:
partition_data()

NUMBER OF SAMPLES
    Class 0:   574    32%
    Class 1:   403    22%
    Class 2:   793    44%
    Total:    1770

PARTITIONED DATA
  Training:    70%, 1239 samples
    Class 0:   409    33%
    Class 1:   268    21%
    Class 2:   562    45%
    Total:    1239

  Validation:  20%, 353 samples
    Class 0:   115    32%
    Class 1:    91    25%
    Class 2:   147    41%
    Total:     353

  Testing:     10%, 178 samples
    Class 0:    50    28%
    Class 1:    44    24%
    Class 2:    84    47%
    Total:     178

  Total:       1770


### Dataset

In [6]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch import optim,nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
from torchtoolbox.transform import Cutout

In [7]:
def default_loader(path):
    return Image.open(path).convert('RGB')

def change_2848_loader(path):
    img = Image.open(path).convert('RGB')
    
    if img.size[0] < 2800:
        return img
    else:
        #print(img.size)###test
        img_pad = np.zeros((img.size[1] + 400, img.size[0], 3), dtype=np.uint8) ###800
        img_pad[400:400+2848, 160:] = np.asanyarray(img)[:, :-160]
        img = Image.fromarray(img_pad)
        return img

class DatasetFromCSV(Dataset):
    def __init__(self, image_root, csv_path, transforms=None, loader=default_loader):###change_2848_loader
        
        self.image_root = image_root
        
        self.data = pd.read_csv(csv_path)
        self.labels = np.asarray(self.data.iloc[:, 1])
        
        # process image name
        imgs = []
        files_names = np.array(self.data.iloc[:, 0])
        for img in files_names:
            imgs.append(os.path.join(self.image_root, str(img)))
        
        self.images = imgs
        self.transforms = transforms
        self.loader = loader
 
    def __getitem__(self, index):

        label = self.labels[index]
        img = self.images[index]
        img = self.loader(img)
        if self.transforms is not None:
            img = self.transforms(img)
        return img, label
    
    def __len__(self):
        return len(self.data.index)

class MyDataset(Dataset):
    def __init__(self, txt, transform=None, target_transform=None, loader=default_loader):
        fh = open(txt, 'r')
        imgs = []
        for line in fh:
            line = line.strip('\n')
            line = line.rstrip('\n')
            words = line.split()
            imgs.append((words[0],int(words[1])))
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = self.loader(fn)
        if self.transform is not None:
            img = self.transform(img)
        return img,label

    def __len__(self):
        return len(self.imgs)

### Transform

In [ ]:
transform_color = [transforms.ColorJitter(brightness=0.5),
                       transforms.ColorJitter(contrast=0.5)
                      ]
transform_affine = [
#                     transforms.RandomAffine(0, shear=20),
                    transforms.RandomAffine(0, scale=(0.95, 1.05)),
                    transforms.RandomAffine(0, translate=(0.05, 0.05)),
                    transforms.RandomAffine(degrees=20)]

train_transform = transforms.Compose([
                transforms.Resize(size=(400), interpolation=Image.LANCZOS), ###460
#                 transforms.CenterCrop(size=400),
                transforms.RandomApply(transform_affine, p=0.5),
                transforms.RandomHorizontalFlip(p=0.5), #随机水平翻转图片
                transforms.RandomApply(transform_color, p=0.5),
#                 Cutout(0.5, scale=(0.02, 0.2), ratio=(0.4, 2.5)),
                transforms.CenterCrop(size=480),
                transforms.ToTensor(), #将图片变成 Tensor，并且把数值normalize到[0,1]    
#                 transforms.Normalize([0.320, 0.197, 0.103], [0.317, 0.206, 0.128])
                        ])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


### 得到均值与方差

In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
import os

train_transform = transforms.Compose([
    transforms.Resize((400, 400)), # 可以改成你图片近似大小或者模型要求大小 ### 224
    transforms.ToTensor(),
])

# train_data= MyDataset("My_data/Training_Set/Training/", "My_data/Training_Set/RFMiD_Training_Labels.csv", train_transform)

# train_loader = torch.utils.data.DataLoader(train_data,batch_size=1000, shuffle=True)

batch_size = 500
root = "/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data"
train_dataset = DatasetFromCSV("/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_org_image/", "/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_labels_train.csv", transforms=train_transform)
valid_dataset = DatasetFromCSV("/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_org_image/", "/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_labels_validation.csv", transforms=train_transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, num_workers=0)

train = iter(train_loader).next()[0]  # 500张图片的mean std
train_mean = np.mean(train.numpy(), axis=(0, 2, 3))
train_std = np.std(train.numpy(), axis=(0, 2, 3))

print(train_mean, train_std)

### Training

In [ ]:
!python '/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/train_efficientnet_album.py'

Namespace(batch_size=16, epochs=100, image_size=400, initial_lr=0.01, n_class=3, num_workers=4, root='/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/', save_path='/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/checkpoints/b0300', train_dir='/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/train_set')
Training......
Train -> Epoch: 000, train_acc: 0.5093, train_loss: 1.2057
Valid -> Epoch: 000, valid_acc: 0.2748, valid_loss: 117.0088
saved model
Training......
Train -> Epoch: 001, train_acc: 0.5529, train_loss: 0.8946
Valid -> Epoch: 001, valid_acc: 0.4363, valid_loss: 2.1169
saved model
Training......
Train -> Epoch: 002, train_acc: 0.6659, train_loss: 0.7299
Valid -> Epoch: 002, valid_acc: 0.4759, valid_loss: 1.5042
saved model
Training......
Train -> Epoch: 003, train_acc: 0.7724, train_loss: 0.5956
Valid -> Epoch: 003, valid_acc: 0.4278, valid_loss: 6.8726
Training......
Train -> Epoch: 004, train_acc: 0.8499, train_loss: 0.4

### Testing

In [11]:
test_dataset = DatasetFromCSV("/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_org_image/", "/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/data_labels_test.csv", transforms=train_transform)

In [23]:
from shutil import copy

In [24]:
for i in range(len(test_dataset)):    
    img_tmp = test_dataset.images[i]
    labels_tmp = test_dataset.labels[i]
    #print(img_tmp, labels_tmp)
    copy(img_tmp, "/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/test_image/")

In [25]:
!python '/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/predict_classification.py'

Namespace(image_size=400, model='/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/checkpoints/b0300/weights/best.pt', source='/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/test_image/')
Load Model Done!!!
6tjqTU40.png -> out: tensor([[-46.9221,  -3.0930,   9.9608]])
6tjqTU40.png -> preds: tensor([[1.9772e-25, 2.1420e-06, 1.0000e+00]])
6tjqTU40.png -> index: tensor([2])-> index.item=2
6tjqTU40.png -> prod: tensor([1.0000])




YQ6ZnSa9.png -> out: tensor([[10.5710, -3.5319, -0.7555]])
YQ6ZnSa9.png -> preds: tensor([[9.9999e-01, 7.5016e-07, 1.2049e-05]])
YQ6ZnSa9.png -> index: tensor([0])-> index.item=0
YQ6ZnSa9.png -> prod: tensor([1.0000])




2VQo3H65.png -> out: tensor([[ 13.1878, -10.8330,   2.1547]])
2VQo3H65.png -> preds: tensor([[9.9998e-01, 3.6974e-11, 1.6158e-05]])
2VQo3H65.png -> index: tensor([0])-> index.item=0
2VQo3H65.png -> prod: tensor([1.0000])




CfPTRl6j.png -> out: tensor([[12.3416, -6.8977,  0.4473]

### 图像分析

#### 训练集

In [ ]:
train_txt = "/content/gdrive/MyDrive/EQ2445/Pytorch_Classification_Intergration/my_data/Master project data/train.txt"
data = open(train_txt, mode='r').readlines()[:36]

In [ ]:
fig = plt.figure(figsize=(14, 30))
for i in range(len(data)):
    ax = fig.add_subplot(9, 4, i+1, xticks=[], yticks=[])
    file_name, label = data[i].rstrip("\n").split(" ")
    image = Image.open(file_name).convert("RGB")
    ax.imshow(image)
    ax.set_title(f"{int(label)+1}")
plt.tight_layout()
plt.show()

<Figure size 1008x2160 with 0 Axes>

#### 测试集

In [ ]:
test_path = "./My_data/Test_set/"
test_images = os.listdir(test_path)
len(test_images)

FileNotFoundError: ignored

In [ ]:
fig = plt.figure(figsize=(14, 30))
for i in range(len(test_images[:36])):
    ax = fig.add_subplot(9, 4, i+1, xticks=[], yticks=[])
    file_name = os.path.join(test_path, test_images[i])
    image = Image.open(file_name).convert("RGB")
    ax.imshow(image)
plt.tight_layout()
plt.show()